In [4]:
import requests 
import pandas as pd
import numpy as np
import sys; sys.path
import datetime
import pandas as pd
import numpy as np 
from datetime import datetime
from datetime import datetime
from tqdm import tqdm
import random
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler

from bitcoin_deep_learning.call_api import ApiCall
from bitcoin_deep_learning.model import LinearRegressionBaselineModel, RnnDlModel,RandomForestReg
from bitcoin_deep_learning.trainer import cv_train, read_result
from bitcoin_deep_learning.cross_val import cross_val, get_cross_XY, cross_val_trade
api = ApiCall()

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
scaler = MinMaxScaler()
train_df= ApiCall().read_local()
for i in range(1,len(list(train_df.columns))-1):
    train_df[list(train_df.columns)[i]] = scaler.fit_transform(np.array(train_df[train_df.columns[1]]).reshape(-1, 1))
train_df

In [ ]:
model = RandomForestReg()
cv_train(model,train_df)
read_result().sort_values(by="date")

In [ ]:
for n_estimators in [1000,1500,3000]:
    for warm_start in  [True,False]:
        for bootstrap in [True,False]:
            for criterion in ["squared_error", "absolute_error", "poisson"]:
                for min_samples_leaf in [1,5,10,20]:
                    model = RandomForestReg(warm_start=warm_start,
                                            n_estimators=n_estimators,
                                           bootstrap=bootstrap,
                                           criterion=criterion,
                                           min_samples_leaf=min_samples_leaf)

                    cv_train(model,train_df)

In [ ]:
X0 = [[i+5,-2*i]for i in range(1001)]
X1 = np.sin(np.arange(1001))

In [ ]:
df = pd.DataFrame(X0)
df["date"] = X1
df.head(20)

In [ ]:
model = LinearRegressionBaselineModel()
#model = RnnDlModel(epochs=10,patience=2)
past_reality, reality,reality_diff, prediction_diff = cross_val_trade(model,df)

In [ ]:
past_reality[0]

In [ ]:
plt.plot(reality[0])

In [ ]:
plt.plot(prediction_diff[0],c="r")
#plt.plot(reality_diff[0])

In [ ]:
model = LinearRegressionBaselineModel()
#model = RnnDlModel(epochs=10,patience=2)
past_reality, reality,reality_diff, prediction_diff = cross_val_trade(model,df)
plt.plot(prediction_diff[0],c="r")
plt.plot(reality_diff[0])

In [ ]:
df

In [ ]:
X_train_list, Y_train_list, X_test_list,Y_test_list = get_cross_XY(df.drop(columns="date"),data=None)

In [ ]:
X_train, Y_train, X_test,Y_test = X_train_list[0], Y_train_list[0], X_test_list[0],Y_test_list[0]

In [ ]:
X_train[0]

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
scaler = MinMaxScaler()
X_train_scaled =  scaler.fit_transform(X_train[:,-1,:])
X_test_scaled = scaler.transform(X_test[:,-1,:])
#X_train_scaled =  (X_train[:,-1,:])
#X_test_scaled = (X_test[:,-1,:])
X_train_scaled

In [ ]:
from sklearn.linear_model import ElasticNet
model = ElasticNet()
model.fit(X_train_scaled,Y_train)
Y_pred = model.predict(X_test_scaled)
plt.plot(Y_pred)
plt.plot(Y_test)

In [ ]:
Y_train

In [ ]:
X_train[:,-1,:]

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = MinMaxScaler()
scaler.fit(X_train[:,-1,:])

In [ ]:
X_train[:,-1,:]

In [ ]:
model = LinearRegressionBaselineModel(alpha=0.1,l1_ratio=0.001)
df = ApiCall().read_local()
X_train_list, Y_train_list, X_test_list,Y_test_list = get_cross_XY(df,data=None)



In [ ]:
cross_val_trade(model,df)

In [ ]:
past_reality, reality,reality_diff, prediction_diff = cross_val_trade(model,df)

In [ ]:
ct = 0
preds = []
computed_reality = []
for past_prices, diffs in zip(past_reality,prediction_diff):
    preds.append(past_prices * diffs+ past_prices)
preds  

In [ ]:
df = api.read_local()
scaler = MinMaxScaler()
#df  =scaler.fit_transform(df.drop(columns="date"))

In [ ]:
df[df.columns[1]].shape

In [ ]:
df

In [ ]:

df = api.read_local()
scaler = MinMaxScaler()
for i in range(1,len(list(df.columns)-2)):
    df[list(df.columns)[i]] = scaler.fit_transform(np.array(df[df.columns[1]]).reshape(-1, 1))

model = LinearRegressionBaselineModel(alpha=0.5,l1_ratio=0.001)
#model = RnnDlModel(epochs=10,patience=2)
past_reality, reality,reality_diff, prediction_diff = cross_val_trade(model,df)
for i in range(10):
    plt.plot(prediction_diff[i],c="r")
    plt.plot(reality_diff[i])
    plt.show()


In [ ]:
df

In [ ]:
fold_score, score= train(model,df)
print(fold_score,score)

In [ ]:
for counter in range(20):
    alpha = 0.015 * random.random()
    l1 = random.random()*0.01
    # Instanciate model
    regression_model = LinearRegressionBaselineModel(alpha = alpha, l1_ratio = l1)
    # Train and Fit data using the crossval
    val_score, score = train(regression_model,df);
    print(val_score,score)
    past_reality, reality,reality_diff, prediction_diff = cross_val_trade(model,df)
    for i in range(len(reality)):
        plt.plot(prediction_diff[i],c="r")
        plt.plot(reality_diff[i])
        plt.show()

In [ ]:
from bitcoin_deep_learning.trainer import read_result
read_result().sort_values(by="date",ascending=False)

In [ ]:
df = api.read_local()
scaler = MinMaxScaler()
for i in range(1,len(list(df.columns))-2):
    df[list(df.columns)[i]] = scaler.fit_transform(np.array(df[df.columns[1]]).reshape(-1, 1))

model = LinearRegressionBaselineModel(alpha=0.5,l1_ratio=0.001)
#model = RnnDlModel(epochs=10,patience=2)
past_reality, reality,reality_diff, prediction_diff = cross_val_trade(model,df)
ALPHA = [0.001,0.005,0.01,0.1,5,1,10,100]
L1 = [0.0001,0.0005,0.001,0.005,0.001,0.005,0.01,0.1,0.5,1]
for alpha in ALPHA :
    for l1 in L1 : 
        # Instanciate model
        regression_model = LinearRegressionBaselineModel(alpha = alpha, l1_ratio = l1)
        # Train and Fit data using the crossval
        val_score, score = train(regression_model,df);
        print(val_score,score)
        past_reality, reality,reality_diff, prediction_diff = cross_val_trade(model,df)
        for i in range(len(reality)):
            plt.plot(prediction_diff[i],c="r")
            plt.plot(reality_diff[i])
            plt.show()

In [ ]:
read_result().sort_values(by="date",ascending=False).head(50)

In [ ]:
df = api.read_local()
scaler = MinMaxScaler()
for i in range(1,len(list(df.columns))-1):
    scaler = MinMaxScaler()
    df[list(df.columns)[i]] = scaler.fit_transform(np.array(df[df.columns[i]]).reshape(-1, 1))

model = LinearRegressionBaselineModel(alpha=0.5,l1_ratio=0.001)
#model = RnnDlModel(epochs=10,patience=2)
past_reality, reality,reality_diff, prediction_diff = cross_val_trade(model,df)
ALPHA = [0.001,0.005,0.01,0.1,5,1,10,100]
L1 = [0.0001,0.0005,0.001,0.005,0.001,0.005,0.01,0.1,0.5,1]
for alpha in ALPHA :
    for l1 in L1 : 
        # Instanciate model
        regression_model = LinearRegressionBaselineModel(alpha = alpha, l1_ratio = l1)
        # Train and Fit data using the crossval
        val_score, score = cv_train(regression_model,df);
        print(val_score,score)
        past_reality, reality,reality_diff, prediction_diff = cross_val_trade(model,df)
        for i in range(len(reality)):
            plt.plot(reality[i])
            plt.plot()
            plt.show()

In [ ]:
x = np.eye(260,3,31)
x.reshape(260,)

In [ ]:
read_result().sort_values(by="mean_score",ascending=False).head(50)

In [ ]:
train_df = ApiCall().read_local()

In [ ]:
read_result(file="CV_trader.csv")

In [ ]:

model = LinearRegressionBaselineModel()
train(model,train_df)

In [ ]:
ROOT_DIR = "/Users/Zalo/code/AlexandreLaizet/bitcoin_deep_learning"


In [ ]:
file_path = os.path.join(ROOT_DIR,
                        "cross_val_data",
                        'CV_trader.csv')
fieldnames = ["name",'fold_score',"mean_score","min_score","max_score",'hyperparams','date',
             "roi_hodler", "sharpe_hodler", "roi_trader", "sharpe_trader","roi_whale", "sharpe_whale",
              "roi_hodler_whale",  "sharpe_hodler_whale", "roi_charles", 
                "sharpe_charles"]
pd.DataFrame(columns=fieldnames).to_csv(file_path,index=False)
read_result(file="CV_trader.csv")

In [ ]:
from bitcoin_deep_learning.metrics import iterate_cross_val_results
iterate_cross_val_results()

In [ ]:
roi_hodler, roi_trader, roi_whale, roi_hodler_whale, roi_charles, sharpe_hodler, sharpe_trader, sharpe_whale, sharpe_hodler_whale, sharpe_charles = iterate_cross_val_results()


In [ ]:
with open(file_path , 'a', newline='') as csvfile:
            #fieldnames = ["name",'fold_score',"mean_score","min_score","max_score",'hyperparams','date']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writerow({"name":model.name, "fold_score":fold_score,
                            "mean_score":score,"min_score":min(fold_score),
                            "max_score":max(fold_score),
                            "hyperparams":model.hyperparams,
                            'date':datetime.now().strftime("%d-%m %H:%M:%S"),
                            "roi_hodler":roi_hodler
                            "sharpe_hodler": sharpe_hodler
                            "roi_trader": roi_trader
                            "sharpe_trader": sharpe_trader
                            "roi_whale": roi_whale
                            "sharpe_whale": sharpe_whale
                            "roi_hodler_whale": roi_hodler_whale
                            "sharpe_hodler_whale": sharpe_hodler_whale
                            "roi_charles": roi_charles
                            "sharpe_charles":sharpe_charles})
            print("Training done")

In [ ]:
train_df= ApiCall().read_local()

In [ ]:
scaler = MinMaxScaler()
train_df= ApiCall().read_local()
for i in range(1,len(list(train_df.columns))-1):
    train_df[list(train_df.columns)[i]] = scaler.fit_transform(np.array(train_df[train_df.columns[1]]).reshape(-1, 1))
train_df

In [ ]:
for warm_start in  [True,False]:
    for max_features in ["auto", "sqrt", "log2"]:
        for bootstrap in [True,False]:
            for criterion in ["squared_error", "absolute_error", "poisson"]:
                    for max_depht in [None,True]:
                        if max_depht :
                            max_depht = np.random.randint(1,1000)
                        print("oui")
                        min_samples_split = random.randint(2,100)
                        min_samples_leaf = random.randint(2,100)
                        model = RandomForestReg(warm_start=warm_start,
                                               max_features=max_features,
                                               bootstrap=bootstrap,
                                               criterion=criterion,
                                               max_depht=max_depht,
                                               min_samples_split=min_samples_split,
                                               min_samples_leaf=min_samples_leaf)

                        cv_train(model,train_df)

In [ ]:
#model = RandomForestReg()
#cv_train(model,train_df)
read_result().sort_values(by="date")

In [ ]:
alpha= 0.2
l1 = 0.001
model = LinearRegressionBaselineModel(alpha=alpha,l1_ratio=l1)
cv_train(model,df)

In [5]:
df = ApiCall().read_local()
for i in range(1,len(list(df.columns))-2):
    scaler = MinMaxScaler()
    df[list(df.columns)[i]] = scaler.fit_transform(np.array(df[df.columns[i]]).reshape(-1, 1))
df

,date,[AVG]_[NH]_mean_hash_rate,[+]_[NH]_Days_Till_Halving,[//]_[AV]_Stock-to-Flow_Ratio,[+]_[NH]_Circulating_Supply,[+]_[NH]_Issuance,[+]_[NH]_Number_of_Active_Addresses,[+]_[NH]_Mean_Block_Interval,[+]_[NH]_Number_of_Transactions,[+]_[NH]_Number_of_Addresses_with_a_Non-Zero_Balance,...,[//]_[AV]_Realized_Profits-to-Value_(RPV)_Ratio,fear_greed_value,[+]_[NH]_Number_of_Addresses_with_Balance_0.01 - 0.1,[+]_[NH]_Number_of_Addresses_with_Balance_0.1 - 1,[+]_[NH]_Number_of_Addresses_with_Balance_1 - 10,[+]_[NH]_Number_of_Addresses_with_Balance_10 - 100,[+]_[NH]_Number_of_Addresses_with_Balance_100 - 1k,[+]_[NH]_Number_of_Addresses_with_Balance_1k - 10k,[+]_[T]_Bitcoin_Price,[%]_Bitcoin_growth_rate_on_Horizon=7
0,2018-02-01,0.007848,0.572909,0.000278,0.000000,0.818750,0.456903,0.166407,0.410971,0.298699,...,0.124186,0.277778,0.184989,0.099227,0.047961,0.223121,0.954140,0.018223,9014.026207,-0.235616
1,2018-02-02,0.012923,0.572218,0.000267,0.000982,0.868750,0.469275,0.178849,0.345245,0.294998,...,0.085755,0.111111,0.173657,0.079729,0.039034,0.214140,0.969427,0.009112,8799.121911,-0.260491
2,2018-02-03,0.008710,0.571527,0.000260,0.001905,0.806250,0.323582,0.177294,0.221320,0.291408,...,0.065849,0.388889,0.163550,0.067367,0.031767,0.197065,0.968153,0.011390,9208.459919,-0.230759
3,2018-02-04,0.017680,0.570836,0.000176,0.002945,0.931250,0.330104,0.122862,0.157196,0.284404,...,0.174428,0.211111,0.135415,0.047554,0.019570,0.195998,0.975796,0.021640,8176.460616,-0.418350
4,2018-02-05,0.023407,0.570145,0.000119,0.004026,0.975000,0.520401,0.105754,0.286112,0.274419,...,0.129880,0.066667,0.108225,0.041632,0.021885,0.194042,0.968577,0.006834,6891.934414,-0.617751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1493,2022-03-05,0.651643,0.541811,0.977461,0.998349,0.203125,0.423504,0.262830,0.313364,0.996192,...,0.009392,0.188889,0.991361,0.988957,0.954228,0.038328,0.057749,0.751708,39416.462688,0.009345
1494,2022-03-06,0.696718,0.541120,0.978999,0.998732,0.228125,0.387928,0.424572,0.258337,0.997347,...,0.013680,0.188889,0.993585,0.989917,0.955602,0.053980,0.054352,0.749431,38423.618936,0.018275
1495,2022-03-07,0.802798,0.540428,0.979046,0.999164,0.281250,0.536329,0.292379,0.404291,0.997544,...,0.033287,0.200000,0.996444,0.993322,0.958043,0.051845,0.049257,0.749431,38018.649505,-0.136405
1496,2022-03-08,0.798383,0.539737,0.979685,0.999597,0.281250,0.575816,0.136858,0.405288,0.996383,...,0.027943,0.177778,0.996800,0.997066,0.961286,0.058070,0.050106,0.757403,38726.302102,-0.145170


In [ ]:
ALPHA = [0.005,0.01,0.1,5,1,10,100]
L1 = [0.0001,0.0005,0.001,0.005,0.001,0.005,0.01,0.1,0.5,1]
for alpha in ALPHA:
    for l1 in L1 : 
        # Instanciate model
        regression_model = LinearRegressionBaselineModel(alpha = alpha, l1_ratio = l1)
        # Train and Fit data using the crossval
        val_score, score = cv_train(regression_model,df);
        print(val_score,score)
        past_reality, reality,reality_diff, prediction_diff = cross_val_trade(model,df)
        for i in range(len(reality)):
            plt.plot(reality[i])
            plt.plot(past_reality[i] * prediction_diff[i]+ past_reality[i])
            plt.plot()
            plt.show()

In [ ]:
ALPHA = [0.001,0.005,0.01,0.1,5,1,10,100]
L1 = [0.0001,0.0005,0.001,0.005,0.001,0.005,0.01,0.1,0.5,1]
for alpha in ALPHA:
    for l1 in L1 : 
        # Instanciate model
        regression_model = LinearRegressionBaselineModel(alpha = alpha, l1_ratio = l1)
        # Train and Fit data using the crossval
        val_score, score = cv_train(regression_model,df.iloc[:,:-1]);
        print(val_score,score)
        past_reality, reality,reality_diff, prediction_diff = cross_val_trade(model,df.iloc[:,:-1])
        for i in range(len(reality)):
            plt.plot(reality_diff[i])
            plt.plot(prediction_diff[i])
            plt.plot()
            plt.show()

In [ ]:
for n_estimators in [1000,1500,3000]:
    for warm_start in  [True,False]:
        for bootstrap in [True,False]:
            for criterion in ["squared_error", "absolute_error", "poisson"]:
                for min_samples_leaf in [1,5,10,20]:
                    model = RandomForestReg(warm_start=warm_start,
                                            n_estimators=n_estimators,
                                           bootstrap=bootstrap,
                                           criterion=criterion,
                                           min_samples_leaf=min_samples_leaf)

                    cv_train(model,df.iloc[:,:-1])
                    print(val_score,score)
                    past_reality, reality,reality_diff, prediction_diff = cross_val_trade(model,df.iloc[:,:-1])
                    for i in range(len(reality)):
                        plt.plot(reality_diff[i])
                        plt.plot(prediction_diff[i])
                        plt.plot()
                        plt.show()

 89%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                | 31/35 [03:04<00:21,  5.32s/it]